## Set UP
## Tools with MCP

In [1]:
import os
from dotenv import load_dotenv

# Load envirinment
load_dotenv()

# Get data from .env file
openai_api_key= os.getenv("OPENAI_API_KEY")
langsmith_api_key=os.getenv("LANGSMITH_API_KEY")
langsmith_tracing=os.getenv("LANGSMITH_TRACING")
langsmith_project=os.getenv("LANGSMITH_PROJECT")
langsmith_endpoint=os.getenv("LANGSMITH_ENDPOINT")

## Load MCP tools
# Connect to the mcp-time server for timezone-aware operations
# This Go-based server provides tools for current time, relative time parsing,
# timezone conversion, duration arithmetic, and time comparison

In [2]:
from langchain_mcp_adapters.client import MultiServerMCPClient
import nest_asyncio

nest_asyncio.apply()

In [4]:
mcp_client = MultiServerMCPClient(
    {
        "time": {
            "transport": "stdio",
            "command": "uvx",
            "args": ["mcp-server-time"],
        }
    },
)

In [8]:
mcp_client = MultiServerMCPClient(
    {
        "mcp": {
            "servers": {
            "time": {
            "command": "uvx",
            "args": ["mcp-server-time"],
        }
        }
            }
    },
)


In [5]:
# Load tools from the MCP server
mcp_tools = await mcp_client.get_tools()
print(f"Loaded {len(mcp_tools)} MCP tools: {[t.name for t in mcp_tools]}")

UnsupportedOperation: fileno

In [ ]:
from langchain.agents import create_agent

agent_with_mcp = create_agent(
    model="openai:gpt-5",
    tools=mcp_tools,
    system_prompt="You are a helpful assistant",
)

In [ ]:
result = await agent_with_mcp.ainvoke(
    {"messages": [{"role": "user", "content": "What's the time in SF right now?"}]}
)
for msg in result["messages"]:
    msg.pretty_print()